In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Resize
from torchvision.io import read_image, ImageReadMode
from vit_pytorch import ViT
from vit_pytorch.extractor import Extractor
import json
import pandas as pd
import random
from collections import Counter
from pathlib import Path
import cv2
import numpy as np
import transformers as hf
from tqdm.auto import tqdm

c:\Users\pachinkomachine\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
GLOBAL_DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
base_path = Path('EXIST 2024 Lab/EXIST 2024 Memes Dataset/training/memes')

In [3]:
with open('EXIST 2024 Lab/EXIST 2024 Memes Dataset/training/EXIST2024_training.json', 'r', encoding='utf-8') as fp:
    annotations = json.load(fp)
df = pd.DataFrame.from_dict(annotations).T
df = df[df['lang'] == 'en'].sample(100)
df.head()

,id_EXIST,lang,text,meme,path_memes,number_annotators,annotators,gender_annotators,age_annotators,ethnicities_annotators,study_levels_annotators,countries_annotators,labels_task4,labels_task5,labels_task6,split
210916,210916,en,A FEMINIST ON THE INTERNET SAID I WAS MANSPLAI...,210916.jpeg,memes/210916.jpeg,6,"[Annotator_644, Annotator_645, Annotator_646, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[White or Caucasian, White or Caucasian, White...","[High school degree or equivalent, Bachelor’s ...","[Italy, Poland, United States, Portugal, Slove...","[YES, YES, YES, YES, NO, YES]","[DIRECT, DIRECT, JUDGEMENTAL, JUDGEMENTAL, -, ...","[[IDEOLOGICAL-INEQUALITY, STEREOTYPING-DOMINAN...",TRAIN-MEME_EN
211858,211858,en,RIHANNA? COCKSUCKER WHORE makeameme.org,211858.jpeg,memes/211858.jpeg,6,"[Annotator_847, Annotator_848, Annotator_849, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[Middle Eastern, Black or African American, Wh...","[High school degree or equivalent, Bachelor’s ...","[South Africa, South Africa, United Kingdom, F...","[YES, YES, YES, YES, YES, YES]","[DIRECT, DIRECT, DIRECT, DIRECT, DIRECT, DIRECT]","[[MISOGYNY-NON-SEXUAL-VIOLENCE], [IDEOLOGICAL-...",TRAIN-MEME_EN
210133,210133,en,What advice do the Mario bros have for dealing...,210133.png,memes/210133.png,6,"[Annotator_479, Annotator_480, Annotator_481, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[White or Caucasian, White or Caucasian, Black...","[Bachelor’s degree, Master’s degree, High scho...","[Canada, France, United Kingdom, Hungary, Unit...","[YES, YES, NO, YES, NO, NO]","[DIRECT, JUDGEMENTAL, -, DIRECT, -, -]","[[STEREOTYPING-DOMINANCE, MISOGYNY-NON-SEXUAL-...",TRAIN-MEME_EN
210646,210646,en,"Hey Girls, did you know that uhmm.. Can go whe...",210646.jpeg,memes/210646.jpeg,6,"[Annotator_586, Annotator_587, Annotator_588, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[Black or African American, White or Caucasian...","[other, Doctorate, Master’s degree, High schoo...","[South Africa, Portugal, United Kingdom, South...","[NO, NO, YES, NO, YES, YES]","[-, -, DIRECT, -, DIRECT, DIRECT]","[[-], [-], [OBJECTIFICATION], [-], [SEXUAL-VIO...",TRAIN-MEME_EN
210291,210291,en,That's right. You get in there & derail that c...,210291.jpeg,memes/210291.jpeg,6,"[Annotator_514, Annotator_515, Annotator_363, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 18-22, 23-45, 46+]","[White or Caucasian, Black or African American...","[Bachelor’s degree, Master’s degree, Master’s ...","[Portugal, Zimbabwe, Mexico, South Africa, Fin...","[NO, NO, NO, NO, YES, NO]","[-, -, -, -, DIRECT, -]","[[-], [-], [-], [-], [MISOGYNY-NON-SEXUAL-VIOL...",TRAIN-MEME_EN


In [4]:
df['number_annotators'].nunique()

1

In [5]:
mini_df = df[['meme', 'text', 'lang', 'ethnicities_annotators', 'labels_task4', 'labels_task5', 'labels_task6']].reset_index(drop=True)
mini_df.head()

,meme,text,lang,ethnicities_annotators,labels_task4,labels_task5,labels_task6
0,210916.jpeg,A FEMINIST ON THE INTERNET SAID I WAS MANSPLAI...,en,"[White or Caucasian, White or Caucasian, White...","[YES, YES, YES, YES, NO, YES]","[DIRECT, DIRECT, JUDGEMENTAL, JUDGEMENTAL, -, ...","[[IDEOLOGICAL-INEQUALITY, STEREOTYPING-DOMINAN..."
1,211858.jpeg,RIHANNA? COCKSUCKER WHORE makeameme.org,en,"[Middle Eastern, Black or African American, Wh...","[YES, YES, YES, YES, YES, YES]","[DIRECT, DIRECT, DIRECT, DIRECT, DIRECT, DIRECT]","[[MISOGYNY-NON-SEXUAL-VIOLENCE], [IDEOLOGICAL-..."
2,210133.png,What advice do the Mario bros have for dealing...,en,"[White or Caucasian, White or Caucasian, Black...","[YES, YES, NO, YES, NO, NO]","[DIRECT, JUDGEMENTAL, -, DIRECT, -, -]","[[STEREOTYPING-DOMINANCE, MISOGYNY-NON-SEXUAL-..."
3,210646.jpeg,"Hey Girls, did you know that uhmm.. Can go whe...",en,"[Black or African American, White or Caucasian...","[NO, NO, YES, NO, YES, YES]","[-, -, DIRECT, -, DIRECT, DIRECT]","[[-], [-], [OBJECTIFICATION], [-], [SEXUAL-VIO..."
4,210291.jpeg,That's right. You get in there & derail that c...,en,"[White or Caucasian, Black or African American...","[NO, NO, NO, NO, YES, NO]","[-, -, -, -, DIRECT, -]","[[-], [-], [-], [-], [MISOGYNY-NON-SEXUAL-VIOL..."


In [6]:
def label4_choose(row):
    # We're going with the simplest option, majority voting
    # Tiebreaker will be random
    label_list = row['labels_task4']
    label_counts = Counter(label_list)
    max_count = max(label_counts.values())
    max_labels = [label for label, count in label_counts.items() if count == max_count]
    if len(max_labels) == 1:  # Majority label is unique
        return max_labels[0]
    else:  # Random tiebreaker
        return random.choice(max_labels)

mini_df['label_task4'] = mini_df.apply(label4_choose, axis=1).map({'YES': 1, 'NO': 0})
mini_df.head()

,meme,text,lang,ethnicities_annotators,labels_task4,labels_task5,labels_task6,label_task4
0,210916.jpeg,A FEMINIST ON THE INTERNET SAID I WAS MANSPLAI...,en,"[White or Caucasian, White or Caucasian, White...","[YES, YES, YES, YES, NO, YES]","[DIRECT, DIRECT, JUDGEMENTAL, JUDGEMENTAL, -, ...","[[IDEOLOGICAL-INEQUALITY, STEREOTYPING-DOMINAN...",1
1,211858.jpeg,RIHANNA? COCKSUCKER WHORE makeameme.org,en,"[Middle Eastern, Black or African American, Wh...","[YES, YES, YES, YES, YES, YES]","[DIRECT, DIRECT, DIRECT, DIRECT, DIRECT, DIRECT]","[[MISOGYNY-NON-SEXUAL-VIOLENCE], [IDEOLOGICAL-...",1
2,210133.png,What advice do the Mario bros have for dealing...,en,"[White or Caucasian, White or Caucasian, Black...","[YES, YES, NO, YES, NO, NO]","[DIRECT, JUDGEMENTAL, -, DIRECT, -, -]","[[STEREOTYPING-DOMINANCE, MISOGYNY-NON-SEXUAL-...",1
3,210646.jpeg,"Hey Girls, did you know that uhmm.. Can go whe...",en,"[Black or African American, White or Caucasian...","[NO, NO, YES, NO, YES, YES]","[-, -, DIRECT, -, DIRECT, DIRECT]","[[-], [-], [OBJECTIFICATION], [-], [SEXUAL-VIO...",1
4,210291.jpeg,That's right. You get in there & derail that c...,en,"[White or Caucasian, Black or African American...","[NO, NO, NO, NO, YES, NO]","[-, -, -, -, DIRECT, -]","[[-], [-], [-], [-], [MISOGYNY-NON-SEXUAL-VIOL...",0


In [7]:
vit = ViT(
    image_size = 256,
    patch_size = 32,
    num_classes = 2,
    dim = 1024,
    depth = 6,
    heads = 16,
    mlp_dim = 2048,
    dropout = 0.1,
    emb_dropout = 0.1
)
vit = Extractor(vit).to(GLOBAL_DEVICE)  # We only need the embeddings from the ViT

# Number of patches, n = (image_size // patch_size) ** 2

In [8]:
bert_tokeniser = hf.BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = hf.BertModel.from_pretrained('bert-base-uncased').to(GLOBAL_DEVICE)

In [9]:
def dataframe_train_test_split(df, target_label, seed=42, test_size=0.2, split_labels=True):
    train = df.sample(frac=(1.0 - test_size), random_state=seed).reset_index(drop=True)
    test = df.drop(train.index).sample(frac=1.0, random_state=seed).reset_index(drop=True)

    if split_labels:
        return train.drop(columns=target_label), test.drop(columns=target_label), train[target_label], test[target_label]
    else:
        return train, test

X_train, X_test, y_train, y_test = dataframe_train_test_split(mini_df[['meme', 'text', 'label_task4']], 'label_task4', test_size=0.2, seed=42)
X_train


,meme,text
0,211991.jpeg,"Ais Women He's not online, he's surely with th..."
1,211784.png,"Random girl: ,,Men today don't know how to tre..."
2,210477.jpeg,"""Hey beautiful, why don't you smile?"" because ..."
3,211346.jpeg,"DURING LABOR, THE PAIN IS SO GREAT THAT A WOMA..."
4,210002.jpeg,"When your homie says ""a woman's body count sho..."
...,...,...
75,210926.jpeg,"""INSERT MANSPLAINING HERE"" imgflip.com ALL WOM..."
76,212000.jpeg,"BITCHES BE UGLY AS SHIT TALKIN BOUT SOME ""IF I..."
77,211646.jpeg,When your stepsister catches you listening to ...
78,211816.jpeg,WHEN YOU FIND YOU 49 YEAR OLD р 12 AU TROPHY W...


In [10]:
class ExistDataset(Dataset):
    def __init__(self, features, labels, img_dir, img_transform=None, caption_transform=None, target_transform=None):
        self.features = features
        self.labels = labels
        self.img_dir = img_dir
        self.img_transform = img_transform
        self.caption_transform = caption_transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_path = str(self.img_dir.joinpath(self.features['meme'].iloc[idx]))
        image = read_image(img_path, mode=ImageReadMode.RGB).to(device=GLOBAL_DEVICE, dtype=torch.float32)
        caption = self.features['text'].iloc[idx]
        label = self.labels.iloc[idx]
        if self.img_transform:
            image = self.img_transform(image)
        if self.caption_transform:
            caption = self.caption_transform(caption)
        if self.target_transform:
            label = self.target_transform(label)
        return image, caption, label

In [11]:
resizer = Resize((256, 256), antialias=True)

def resize_images(img_tensor):
    return resizer(img_tensor)

In [12]:
train_dataset = ExistDataset(X_train, y_train, base_path, img_transform=resize_images)
train_dataset

In [13]:
max_length = 512

In [14]:
class ConcatArch(nn.Module):
    def __init__(self, hidden_size, num_classes):
        super().__init__()
        self.hidden_size = hidden_size
        self.fc1 = nn.Linear(768 + 1024, self.hidden_size)
        self.fc2 = nn.Linear(self.hidden_size, num_classes)
        self.activation = nn.ReLU()
        self.output = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        output = self.output(x)
        return output


In [16]:

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)

model = ConcatArch(hidden_size=128, num_classes=1).to(GLOBAL_DEVICE)

optim = hf.AdamW(model.parameters(), lr=0.001)
loss_fn = nn.BCELoss()
scaler = torch.cuda.amp.GradScaler(enabled=True)

epochs = 10

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    for image, caption, label in tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{epochs}'):
        optim.zero_grad()
        
        train_tokens = bert_tokeniser.batch_encode_plus(
            list(caption),
            padding='max_length',
            max_length=max_length,
            truncation=True
        )

        train_seq = torch.tensor(train_tokens['input_ids'], device=GLOBAL_DEVICE)
        train_mask = torch.tensor(train_tokens['attention_mask'], device=GLOBAL_DEVICE)

        label = label.to(device=GLOBAL_DEVICE, dtype=torch.float32)

        with torch.cuda.amp.autocast(enabled=True):
            _, cls_caption = bert_model(train_seq, attention_mask=train_mask, return_dict=False)
            _, embeddings = vit(image)
            cls_img = embeddings[:, 0, :]
            cls = torch.cat([cls_caption, cls_img], dim=1)
            output = model(cls)
        
        loss = loss_fn(output, label.unsqueeze(dim=1))
        loss.backward()
        optim.step()
        train_loss += loss.item()

        train_loss /= len(train_dataloader)

        print(f'Train Loss: {train_loss:.4f}')
        torch.cuda.empty_cache()

c:\Users\pachinkomachine\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/10:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1/10:  10%|█         | 1/10 [00:00<00:06,  1.41it/s]

Train Loss: 3.7500


Epoch 1/10:  20%|██        | 2/10 [00:01<00:03,  2.09it/s]

Train Loss: 7.8750


...

Train Loss: 2.7796


Epoch 10/10:  90%|█████████ | 9/10 [00:02<00:00,  3.76it/s]

Train Loss: 6.5280


Epoch 10/10: 100%|██████████| 10/10 [00:02<00:00,  3.79it/s]

Train Loss: 3.1528
